In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import json
import os
import pandas as pd

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
print(model_id)

In [ ]:
HF_TOKEN='please enter your hf token'

In [ ]:
df = pd.read_csv('/kaggle/input/train-csv/train_data.csv')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token= HF_TOKEN)

In [ ]:
df

In [ ]:
df['id']=df.apply(lambda row: row['Technique']  if row['ID'] =="No sub-Techniques" else row['ID'],axis=1)

In [ ]:
df

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=HF_TOKEN
)

In [ ]:
df.groupby('id')

In [ ]:
df

In [ ]:
 pad_token_id=tokenizer.eos_token_id

In [ ]:
# Create a new column for the new descriptions
result=[]
a=0
# Iterate over the first 10 rows
for index, row in df.iterrows():
    technique_id = row['id']
    tactic_name = row['Tactic-Name']
    technique_name = row['Technique-Name']
    description = row['Description']
    
    prompt = (
        f"Generate a paragraph of 5 threat-action sentences for the technique with ID = {technique_id}, "
        f"tactic name = {tactic_name}, and technique name = {technique_name}, and Description = {description}. "
        "Ensure that the sentences are diverse and do not replicate descriptions from the MITRE ATT&CK framework."
    )
    
    messages = [
        {"role": "system", "content": "You are a cybersecurity expert tasked with generating unique and varied technique descriptions."},
        {"role": "user", "content": prompt}
    ]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    
    outputs = model.generate(
        input_ids,
        max_new_tokens=2056,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    response = outputs[0][input_ids.shape[-1]:]
    generated_text = tokenizer.decode(response, skip_special_tokens=True)
    
    # Add the generated text to the new column in the DataFrame
    result.append({
        "id": technique_id,
        "tactic_name": tactic_name,
        "technique_name": technique_name,
        "description": description,
        "generated_sentences": generated_text
    })
    print(a)
    a+=1
results_df = pd.DataFrame(result)
results_df.to_csv('generated_threat_action_sentences.csv', index=False)    

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
torch.cuda.empty_cache()